<a href="https://colab.research.google.com/github/arjasc5231/moodots/blob/main/SER/CNN/CNNonMFCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
learning_rate = 0.001
training_epochs = 60
batch_size = 100

In [ ]:
mel_X_train, mel_X_test, mel_Y_train, mel_Y_test = np.load("/content/drive/MyDrive/team_runner/colab/dataset/emoDB/emo_mfcc_more.npy", allow_pickle=True)

print(mel_X_train.shape)
print(mel_Y_train.shape)
print(mel_X_test.shape)
print(mel_Y_test.shape)

(683, 20, 128)
(683,)
(228, 20, 128)
(228,)


In [ ]:
mel_X_train = np.expand_dims(mel_X_train, axis=-1)
mel_X_test = np.expand_dims(mel_X_test, axis=-1)

mel_Y_train = to_categorical(mel_Y_train, 7)
mel_Y_test = to_categorical(mel_Y_test, 7)    

mel_train_dataset = tf.data.Dataset.from_tensor_slices((mel_X_train, mel_Y_train)).shuffle(
                buffer_size=100000).batch(batch_size)
mel_test_dataset = tf.data.Dataset.from_tensor_slices((mel_X_test, mel_Y_test)).batch(batch_size)


In [ ]:
def create_model():
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu, padding='SAME', 
                                  input_shape=(20, 128, 1)))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(256, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(64, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(7))
    return model

In [ ]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 20, 128, 32)       320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 32, 64)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 32, 128)        73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 16, 128)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6144)              0

In [ ]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(
        y_pred=logits, y_true=labels, from_logits=True))    
    return loss

In [ ]:
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [ ]:
def evaluate(model, images, labels):
    logits = model(images, training=False)
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

In [ ]:
optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0
    
    for images, labels in mel_train_dataset:
        grads = grad(model, images, labels)                
        optimizer.apply_gradients(zip(grads, model.variables))
        loss = loss_fn(model, images, labels)
        acc = evaluate(model, images, labels)
        avg_loss = avg_loss + loss
        avg_train_acc = avg_train_acc + acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in mel_test_dataset:        
        acc = evaluate(model, images, labels)        
        avg_test_acc = avg_test_acc + acc
        test_step += 1    
    avg_test_acc = avg_test_acc / test_step    

    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))

print('Learning Finished!')

Learning started. It takes sometime.
Epoch: 1 loss = 0.41242963 train accuracy =  0.9354 test accuracy =  0.4824
Epoch: 2 loss = 0.43618998 train accuracy =  0.9111 test accuracy =  0.4790
Epoch: 3 loss = 0.47035724 train accuracy =  0.9168 test accuracy =  0.4762
Epoch: 4 loss = 0.44763574 train accuracy =  0.8985 test accuracy =  0.4776
Epoch: 5 loss = 0.45173672 train accuracy =  0.9254 test accuracy =  0.4571
Epoch: 6 loss = 0.45888716 train accuracy =  0.9277 test accuracy =  0.4824
Epoch: 7 loss = 0.44646755 train accuracy =  0.9362 test accuracy =  0.4638
Epoch: 8 loss = 0.47105995 train accuracy =  0.8979 test accuracy =  0.5029
Epoch: 9 loss = 0.44951898 train accuracy =  0.9303 test accuracy =  0.4862
Epoch: 10 loss = 0.48037454 train accuracy =  0.9185 test accuracy =  0.4924
Epoch: 11 loss = 0.43275520 train accuracy =  0.9219 test accuracy =  0.4910
Epoch: 12 loss = 0.47037092 train accuracy =  0.9268 test accuracy =  0.4676
Epoch: 13 loss = 0.49097794 train accuracy =  0.

In [ ]:
conf_mat = [[0]*7 for i in range(7)] #mat[real_label]=predicted_label list

for images, labels in mel_test_dataset:
  logits = model(images, training=False)
  logits_max = tf.math.argmax(logits,1)
  labels_max = tf.math.argmax(labels,1)
  for i in range(len(logits)): conf_mat[logits_max[i]][labels_max[i]]+=1

for i in range(7): print(conf_mat[i])

[49, 1, 2, 3, 12, 0, 0]
[0, 27, 0, 0, 1, 5, 4]
[1, 4, 21, 1, 3, 0, 0]
[2, 3, 0, 3, 0, 0, 0]
[0, 0, 0, 3, 9, 0, 3]
[0, 3, 1, 2, 2, 33, 1]
[1, 4, 1, 4, 1, 0, 18]


In [ ]:
conf_mat_normal = []
for i in range(7):
  s = sum(conf_mat[i])
  conf_mat_normal.append(list(map(lambda x:(x/s)*100, conf_mat[i])))

label = ['anger','boredom','disgust','fear','happy','sad','neutral']
print('\t'+'\t'.join(label))
for i in range(7):
  print(label[i], end='')
  for j in range(7): print('\t%2.0f'%conf_mat_normal[i][j], end=' ')
  print()
# fear의 데이터가 8개밖에 없다. -> kfold해봄

	anger	boredom	disgust	fear	happy	sad	neutral
anger	73 	 1 	 3 	 4 	18 	 0 	 0 
boredom	 0 	73 	 0 	 0 	 3 	14 	11 
disgust	 3 	13 	70 	 3 	10 	 0 	 0 
fear	25 	38 	 0 	38 	 0 	 0 	 0 
happy	 0 	 0 	 0 	20 	60 	 0 	20 
sad	 0 	 7 	 2 	 5 	 5 	79 	 2 
neutral	 3 	14 	 3 	14 	 3 	 0 	62 
